In [ ]:
!pip install -U datasets evaluate transformers
!pip install -q evaluate
!pip install -q rouge_score
!pip install tree-sitter-python==0.21
!pip install codebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [ ]:
from datasets import load_dataset
from evaluate import load
import re
import json
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from transformers import (AutoTokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Trainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, GPT2LMHeadModel,
                          DataCollatorForLanguageModeling, TrainingArguments,EvalPrediction)
import os
import pickle
from tqdm import tqdm
from codebleu import calc_codebleu
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1. Load model and tokenizer

In [ ]:
all_model_locs = {
    "T5-pretrained": "Salesforce/codet5-base",
    "T5-finetuned": '/content/drive/MyDrive/NLP_Code_Generation/t5-base-codeparrots', ## replace this with your fine-tuned model
    "GPT2-pretrained": "gpt2",
    "GPT2-finetuned": '/content/drive/MyDrive/NLP_Code_Generation/gpt2-codeparrots',  ## replaced this with your fidne-tuned model
}
### dictionary storing the model and tokenizer for each model for evaluation
all_models = {}
for model_name, model_loc in all_model_locs.items():
  all_models[model_name] = {}
  all_models[model_name]["tokenizer"] = AutoTokenizer.from_pretrained(model_loc)

  if model_name.startswith("T5"):
    all_models[model_name]["model"] = T5ForConditionalGeneration.from_pretrained(model_loc)

  elif model_name.startswith("GPT2"):
      all_models[model_name]["tokenizer"].pad_token = all_models[model_name]["tokenizer"].eos_token
      all_models[model_name]["model"] = GPT2LMHeadModel.from_pretrained(model_loc)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

t5_tokenizer = all_models["T5-finetuned"]["tokenizer"]
t5_model_pretrained = all_models["T5-pretrained"]["model"].to(device)
t5_model_finetuned = all_models["T5-finetuned"]["model"].to(device)

gpt_tokenizer = all_models["GPT2-finetuned"]["tokenizer"]
gpt_model_pretrained = all_models["GPT2-pretrained"]["model"].to(device)
gpt_model_finetuned = all_models["GPT2-finetuned"]["model"].to(device)

### 2. Run Python auto coder

In [ ]:
print("================ QUESTION =================")
question = "Write Python code to add two numbers a and b"
print(question)

print("===================T5 pretrained solution ==================")
t5_tokenized_question = t5_tokenizer(question, return_tensors="pt")
t5_pretrained_preds = t5_model_pretrained.generate(t5_tokenized_question['input_ids'].to(device), attention_mask=t5_tokenized_question['attention_mask'].to(device), max_new_tokens=128)[0]
t5_pretrained_decoded_preds = t5_tokenizer.decode(t5_pretrained_preds, skip_special_tokens=True)
print(t5_pretrained_decoded_preds)

print("===================T5 finetuned solution ==================")
t5_finetuned_preds = t5_model_finetuned.generate(t5_tokenized_question['input_ids'].to(device), attention_mask=t5_tokenized_question['attention_mask'].to(device), max_new_tokens=128)[0]
t5_finetuned_decoded_preds = t5_tokenizer.decode(t5_finetuned_preds, skip_special_tokens=True)
print(t5_finetuned_decoded_preds)

# get gpt2 predictions
print("===================GPT finetuned solution ==================")
gpt_tokenized_question = gpt_tokenizer(question, return_tensors="pt")
gpt_finetuned_preds = gpt_model_finetuned.generate(gpt_tokenized_question['input_ids'].to(device), attention_mask=gpt_tokenized_question['attention_mask'].to(device), max_new_tokens=128)[0]
gpt_finetuned_decoded_preds = gpt_tokenizer.decode(gpt_finetuned_preds[len(gpt_tokenized_question['input_ids']):], skip_special_tokens=True)
print(gpt_finetuned_decoded_preds)

================ QUESTION =================
Write a function to calculate add two numbers a and b
===================T5 pretrained solution ==================
 public function add (
===================T5 finetuned solution ==================


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def get_add(a, b):
    return a + b
===================GPT finetuned solution ==================
 a function to calculate add two numbers a and b.

The function should return the sum of the two numbers.

Example:

```python
add_two(1, 2) # => 2
add_two(1, 2) # => 2
```
[CODE]
def add_two(a, b):
    return sum(a*b) + sum(a*b)

def add_two(a, b):
    return sum(a*b) + sum(a*b)

def add_two(a, b):
    return
